# mapboxgl-jupyter

Testing [mapboxgl-jupyter](https://github.com/mapbox/mapboxgl-jupyter/) for creating maps in Jupyter notebooks. The repo itself provides some nice [examples](https://github.com/mapbox/mapboxgl-jupyter/tree/master/examples/notebooks).

In [ ]:
import geojson
import geopandas as gpd
import os
import requests
import zipfile

from mapboxgl.viz import CircleViz, HeatmapViz
from mapboxgl.utils import create_color_stops

In [ ]:
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Download and cache data

Get Shapefile of health sites in Colombia and prepare a GeoJSON for mapping.

In [ ]:
data_url = 'https://healthsites.io/media/shapefiles/Colombia_shapefile.zip'
data_path = 'data/colombia.zip'

if not os.path.exists(data_path):
    r = requests.get(data_url)
    with open(data_path, 'wb') as f:
        f.write(r.content)        

    with zipfile.ZipFile(data_path, 'r') as f:
        f.extractall(os.path.splitext(data_path)[0])    

data_gdf_all_columns = gpd.read_file('data/colombia/colombia.shp')
data_gdf = data_gdf_all_columns[['name', 'type', 'geometry']]
data = geojson.loads(data_gdf.to_json())

## Color health sites by type

In [ ]:
lon_colombia, lat_colombia = (-74.2973, 4.5709)

In [ ]:
category_color_stops = [
    ['hospital', 'rgb(211, 47, 47)'],  
    ['clinic', 'rgb(2, 136, 209)'], 
]

viz = CircleViz(data,
                access_token=token,
                radius=2,
                color_property='type',
                color_default='grey',
                color_function_type='match',
                color_stops=category_color_stops,
                center=(lon_colombia, lat_colombia),
                zoom=4)
viz.show() 

## Heatmap

In [ ]:
heatmap_color_stops = create_color_stops([0.01, 0.25, 0.5, 0.75, 1], colors='OrRd')
heatmap_radius_stops = [[0, 1], [15, 20]]  # increase radius with zoom

viz = HeatmapViz(data, 
                 access_token=token,
                 weight_stops=[[1, 1]],
                 color_stops=heatmap_color_stops,
                 radius_stops=heatmap_radius_stops,
                 opacity=0.8,
                 center=(lon_colombia, lat_colombia),
                 zoom=4)
viz.show()